[주제] 테크 뉴스레터 작성하기

# CrewAI와 Tool 결합

In [13]:
import os
import warnings
warnings.filterwarnings('ignore')


In [14]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# 라이브러리 임포트

### crewai_tools
- SerperDevTool : Serper api를 활용한 결과물 받는 도구, 구글 웹검색 도구
- WebsiteSearchTool : 특정 키워드와 특정 웹사이트를 주면, 해당 웹 사이트에서 키워드와 관련이 있는 부부만 가져옴.
- ScrapeWebsiteTool : 어떤 URL을 주면 해당 url에 있는 텍스트를 스크래핑해서 가져오는 도구

In [15]:
from crewai import Agent, Task, Crew
from crewai.process import Process
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool
from langchain_openai import ChatOpenAI

# 필요한 객체 생성

In [16]:
# llm 객체
llm = ChatOpenAI(model="gpt-4o-mini")

# Tool
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
scrap_tool = ScrapeWebsiteTool()

# Agent 정의

In [17]:
# 테크 트렌드 연구원
researcher = Agent(
    role = "테크 트렌드 연구원",
    goal="인공지능 분야의 최신 기술 트렌드를 한국어로 제공합니다. 지금은 2025년 6월입니다.",
    backstory="기술 트렌드에 예리한 안목을 지닌 전문 분석가이자 AI 개발자입니다.",
    tools=[search_tool, web_rag_tool],
    verbose=True,
    max_iter=5,  # 해당 Agent가 주어진 task를 수행할 때 반복 실행할 횟수를 제한하는 것임(웹검색을 5번 실행, 비용과 연결되어 있음.)
    llm=llm
)

- allow_delegation=False
    - 해당 agnet가 task를 수행할 때 혼자 힘으로 하기 힘들 때 다른 agnet에 위임하는 설정
    - detault=True
    - False로 하지 않으면 수행중에 researcher에게 작업을 넘기는 경우가 발생하는데 이것을 발생하지 않도록 제한하기 위함

In [18]:
# 뉴스레터 작성 agnet
writer = Agent(
    role="뉴스레터 작성자",
    goal="최신 AI 기술 트렌드에 대한 매력적인 테크 뉴스레터를 한국어로 작성하세요. 지금은 2025년 6월입니다.",
    backstory="기술에 대한 열정을 가진 숙련된 작가입니다.",
    verbose=True,
    allow_delegation=False,  
    llm=llm
)

# Task 정의

In [19]:
research = Task(
    description="AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.",
    expected_output="AI 업계에서 가장 주목받는 3대 기술 동향과 그 중요성에 대한 신선한 관점을 요약한 글",
    agent=researcher
)

In [20]:
write = Task(
    description="""
테크 트렌드 연구원의 요약을 바탕으로 AI 산업에 대한 매력적인 테크 뉴스레터를 작성하세요.
테크 뉴스레터이므로 전문적인 용어를 사용해도 괜찮습니다.
""",
    expected_output="최신 기술 관련 소식을 재밌는 말투로 소개하는 4문단짜리 마크다운 형식 뉴스레터", # 결과물 형식
    agent=writer,
    output_file=r"./new_post.md" # the final blog post will be saved here
)

# Crew 정의

In [21]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=False,
    process=Process.sequential,
    output_log_file="logs/crew_run.json",   # 👈 JSON 형태로 모든 스텝이 저장됩니다.
)

# Crew tasks 실행

In [22]:
result = crew.kickoff()

# Agent: 테크 트렌드 연구원
## Task: AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.


# Agent: 테크 트렌드 연구원
## Thought: Action: Search the internet with Serper
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"AI industry latest technology trends 2025\"}"
## Tool Output: 
{'searchParameters': {'q': 'AI industry latest technology trends 2025', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': "6 AI trends you'll see more of in 2025 - Microsoft News", 'link': 'https://news.microsoft.com/source/features/ai/6-ai-trends-youll-see-more-of-in-2025/', 'snippet': 'In 2025, AI will evolve from a tool for work and home to an integral part of both. AI-powered agents will do more with greater autonomy and help simplify your ...', 'position': 1}, {'title': "What's next for AI in 2025 | MIT Technology Review", 'link': 'https://www.technologyreview.com/2025/01/08/1109188/whats-next-for-ai-in-2025/', 'snippet': "What's next for AI in 2025 · 1. Generative virtual playgro

# 마크다운으로 표시

In [23]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

# 2025년 AI 기술 트렌드 뉴스레터

안녕하세요, AI 기술에 대한 최신 소식을 전해드리는 뉴스레터에 오신 것을 환영합니다! 2025년, 인공지능(AI) 업계는 그 어느 때보다도 흥미진진한 변화의 한가운데에 있습니다. 오늘은 특히 주목할 만한 세 가지 기술 동향에 대해 알아보겠습니다. 자, 그럼 두근두근한 AI의 세계로 떠나볼까요?

## 에이전틱 AI: 자율의 시대를 열다!  
에이전틱 AI는 이제 단순한 질문-응답을 넘어, 독립적으로 업무를 수행할 수 있는 자율적인 머신 에이전트로 진화했습니다. 기업들은 이 신기술을 통해 인간의 개입 없이도 복잡한 작업을 자동으로 처리할 수 있는 시스템을 구축하고 있습니다. 이를 통해 인력 자원을 최적화하고, 비용을 절감하면서도 업무 효율성을 극대화할 수 있죠. 마치 AI가 기업의 새로운 직원을 만든 듯한 느낌입니다! 잡다한 업무는 AI가 처리하고, 인간은 더 창의적인 일에 집중할 수 있는 시대가 다가오고 있습니다.

## 소규모 언어 모델: 맞춤형 AI의 강력한 힘  
최근 대형 언어 모델(LLM)의 높은 비용과 처리 속도 문제를 해결하기 위해 소규모 언어 모델이 각광받고 있습니다. 이들은 특정 산업이나 기업의 요구에 맞춰 쉽게 커스터마이즈할 수 있어, 효율성과 정확성을 동시에 제공합니다. 고객 서비스 개선 및 내부 업무 자동화에 기여할 수 있는 이 모델들은 마치 맞춤형 양복처럼, 기업의 니즈에 딱 맞게 설계된 AI 솔루션이 될 것입니다. AI와 함께라면 고객의 마음을 사로잡는 것은 시간 문제일지도 모르겠네요!

## 추론 AI: 인간의 사고를 닮다  
추론 AI는 인간과 유사한 수준의 문제 해결 능력을 갖춘 AI 모델 개발을 목표로 하고 있습니다. 이 기술은 복잡한 문제를 해결하는 데 뛰어난 성능을 발휘하며, 특정 분야에서는 박사급 수준의 성과를 보일 수 있습니다. 이러한 발전은 기업들이 프로젝트 계획 및 의사 결정에 있어 AI를 적극적으로 활용하는 계기가 될 것입니다. AI가 인간의 의도를 이해하고 이를 기반으로 작업을 수행하게 된다면, 기업의 경쟁력은 더욱 강화될 것이고, 여러분의 비즈니스도 한층 더 발전할 수 있을 것입니다.

이 세 가지 기술 동향은 2025년 AI 업계의 미래를 밝히는 나침반과도 같습니다. 기업들이 AI를 통해 혁신적인 변화를 이루어 나가며, 기술은 단순히 발전하는 것을 넘어 사회 전반에 걸쳐 큰 변화를 가져올 것입니다. 앞으로도 AI와 함께하는 여정이 기대되네요! 다음 뉴스레터에서 더 흥미로운 소식으로 찾아뵙겠습니다. 감사합니다!